In [1]:
# stdlib imports
from datetime import datetime, timedelta
from time import time

# Third party imports
import pandas as pd

# Local imports
from libcomcat.dataframes import get_summary_data_frame
from libcomcat.search import (get_event_by_id, get_authoritative_info, search)

import plotly.express as px
import plotly.graph_objects as go

In [123]:
df = get_summary_data_frame(search(starttime=datetime.now()-timedelta(days=365*10), 
                                    endtime=datetime.now(),
                                    minlatitude=35, 
                                    maxlatitude=72, 
                                    minlongitude=-25, 
                                    maxlongitude=45))

In [126]:
df.to_json("eu_earthquakes_decade.json")

In [51]:
bins = [(0,4),(4,6),(6,7),(7,10)]
colors = ["lightgrey","#ffffbf","#fdae61","#d7191c"]
fig = go.Figure()

for i in range(len(bins)):
    sizeBin = bins[i]
    df_sub = df[(df["magnitude"] > sizeBin[0]) & (df["magnitude"] <= sizeBin[1])]

    fig.add_trace(go.Scattergeo(
        lon=df_sub["longitude"],
        lat=df_sub["latitude"],
        text=df_sub["location"],
        marker= dict(
            size = df_sub["magnitude"]**2,
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'            
        ),
        name = '{0} < x <= {1}'.format(sizeBin[0],sizeBin[1])))
    
fig.update_layout(
        title_text = '2022 US Seismic activity<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope = 'north america',
            landcolor = 'rgb(217, 217, 217)',
            fitbounds= "locations",
        )
        
    )

fig.update_geos(
            showland=True,
            showcountries=True,
            showsubunits=True,
            subunitcolor="Blue",
)


fig.show()

In [6]:
df.to_json("us_earthquakes_decade_minmag_25.json")

In [57]:
df = pd.read_json("us_earthquakes_decade.json")

In [128]:
coordinates

[(-124.96936776663334, 49.96239905619577),
 (-65.09816556666667, 49.96239905619577),
 (-65.09816556666667, 24.641345510185314),
 (-124.96936776663334, 24.641345510185314)]

In [135]:
import datashader as ds
from pyproj import Transformer
from colorcet import fire
import datashader.transfer_functions as tf

t3857_to_4326 = Transformer.from_crs(3857, 4326, always_xy=True)
df.loc[:, "longitude_3857"], df.loc[:, "latitude_3857"] = ds.utils.lnglat_to_meters(
    df.longitude, df.latitude
)

RESOLUTION = 1000
cvs = ds.Canvas(plot_width=RESOLUTION, plot_height=RESOLUTION)
agg = cvs.points(df, x="longitude_3857", y="latitude_3857")

coordinates =[t3857_to_4326.transform(
                        agg.coords["longitude_3857"].values[a],
                        agg.coords["latitude_3857"].values[b],
                    )
                    for a, b in [(0, -1), (-1, -1), (-1, 0), (0, 0)]]

center_x = coordinates[0][0] + (coordinates[1][0] - coordinates[0][0])/2
center_y = coordinates[0][1] + (coordinates[-1][1] - coordinates[0][1])/2

img = tf.shade(agg, cmap=fire).to_pil()
fig = go.Figure(go.Scattermapbox())
fig.update_layout(
    mapbox={
        "style": "carto-darkmatter",
        "center": go.layout.mapbox.Center(
                    lon=center_x,
                    lat=center_y
                ),
        "zoom":2.4,
        "layers": [
            {
                "sourcetype": "image",
                "source": img,
                "coordinates": coordinates
            }
        ],
    },
    margin={"l": 0, "r": 0, "t": 0, "b": 0},
)
fig.show()


In [136]:
import plotly.express as px

magnitude_count = df[df["magnitude"] >0]["magnitude"].round(1).value_counts().reset_index()
magnitude_count.columns = ["magnitude", "count"]


fig = px.scatter(magnitude_count, x="magnitude", y="count" , log_y=True, template="plotly_dark")
fig.update_layout(

)
fig.show()

,id,time,location,latitude,longitude,depth,magnitude,alert,url,eventtype,significance,longitude_3857,latitude_3857
0,nn00395225,1354284377487,"37 km SSE of Goldfield, Nevada",37.382100,-117.116000,6.700000,0.90,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,12,-1.303729e+07,4.492501e+06
1,ci15253249,1354284404320,"9km NE of Coso Junction, CA",36.094167,-117.866667,6.112000,1.58,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,38,-1.312086e+07,4.313586e+06
2,nc71896501,1354284582660,"5 km SSW of Cobb, California",38.779500,-122.756333,0.713000,1.27,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,25,-1.366517e+07,4.690136e+06
3,ci15253257,1354285729550,"4km NNW of Lake Henshaw, CA",33.276500,-116.773667,3.847000,1.08,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,18,-1.299919e+07,3.932062e+06
4,ci15253265,1354285947700,"24km SW of Ocotillo Wells, CA",33.004167,-116.333000,7.632000,2.25,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,78,-1.295013e+07,3.895857e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
759156,nc73812326,1669638773280,"8km ENE of Covelo, CA",39.806500,-123.159332,6.950000,2.42,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,90,-1.371003e+07,4.837863e+06
759157,nc73812331,1669638812770,"3km WSW of Cobb, CA",38.815166,-122.761498,1.640000,1.72,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,46,-1.366575e+07,4.695230e+06
759158,ci40381872,1669639686800,"20km ENE of Big Bear City, CA",34.327167,-116.637000,0.010000,1.51,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,35,-1.298397e+07,4.072818e+06
759159,ok2022xhta,1669640002022,"9 km WSW of Minco, Oklahoma",35.288296,-98.042030,4.473833,1.76,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,48,-1.091399e+07,4.203129e+06
